In [1]:
import pandas as pd
import numpy as np

Load the required libraries, as well as the dataframe we will work with from robust_alternative.csv

In [2]:
robust_alternative = pd.read_csv('Downloads/robust_alternatives.csv')

In [3]:
robust_alternative = robust_alternative.drop(['Unnamed: 0'],axis = 1)

In [4]:
robust_alternative

,income,incQ,dti,region,SOC_label,any_late,age_group,income quintile
0,47387.57,8,0.235342,South West,Professional occupations,0,Aged 35 to 49 years,4
1,66577.07,10,0.000000,East,"Managers, directors and senior officials",0,Aged 65 years and over,5
2,33240.98,6,0.000000,South East,Professional occupations,0,Aged 50 to 64 years,3
3,41529.33,7,0.036065,South West,Professional occupations,1,Aged 35 to 49 years,4
4,56176.13,9,0.000000,South East,Professional occupations,0,Aged 25 to 34 years,5
...,...,...,...,...,...,...,...,...
99995,32075.79,5,0.016850,South West,Skilled trades,1,Aged 35 to 49 years,3
99996,44287.21,8,0.000000,London,Professional occupations,1,Aged 35 to 49 years,4
99997,51767.80,9,0.000000,East,Professional occupations,0,Aged 50 to 64 years,5
99998,16043.91,2,0.000000,Wales,Professional occupations,0,Aged 35 to 49 years,1


Perform a hybrid approach for sampling housing tenure label (IPF for joint age group, income quintile, region, housing tenure), followed by Bayesian imputation based on debt-to-income and late payment probability. 

In [5]:
import pandas as pd
import numpy as np

# ─────────────────────────────────────────────────────────────────────────────
# 1) Load marginal data from CSVs (file paths updated)
# ─────────────────────────────────────────────────────────────────────────────

# 1.a) Age × Tenure
df_age = pd.read_csv('Downloads/mortgage x age.csv')
# Columns: 'age group', 'tenure label', 'frequency'
age_labels = sorted(df_age['age group'].unique())
tenure_labels = sorted(df_age['tenure label'].unique())
age_tenure_counts = (
    df_age
    .pivot(index='age group', columns='tenure label', values='frequency')
    .reindex(index=age_labels, columns=tenure_labels, fill_value=0)
    .values
)  # shape: (len(age_labels), len(tenure_labels))

# 1.b) Income Quintile × Tenure
df_inc = pd.read_csv('Downloads/mortgage x income.csv')
# Columns: 'income quintile', 'tenure label', 'frequency'
inc_labels = sorted(df_inc['income quintile'].unique())
inc_tenure_counts = (
    df_inc
    .pivot(index='income quintile', columns='tenure label', values='frequency')
    .reindex(index=inc_labels, columns=tenure_labels, fill_value=0)
    .values
)  # shape: (len(inc_labels), len(tenure_labels))

# 1.c) Region × Tenure
df_region = pd.read_csv('Downloads/mortgage x region.csv')
# Columns: 'region', 'tenure label', 'frequency'
region_labels = sorted(df_region['region'].unique())
region_tenure_counts = (
    df_region
    .pivot(index='region', columns='tenure label', values='frequency')
    .reindex(index=region_labels, columns=tenure_labels, fill_value=0)
    .values
)  # shape: (len(region_labels), len(tenure_labels))

# 1.d) Debt-to-Income per Tenure (average + count)
df_dti = pd.read_csv('Downloads/mortgage x debt-to-income.csv')
# Columns: 'tenure label', 'debt-to-income', 'Observations'
dti_means = (
    df_dti
    .set_index('tenure label')
    ['debt-to-income']
    .reindex(tenure_labels)
    .values
)
dti_counts = (
    df_dti
    .set_index('tenure label')
    ['Observations']
    .reindex(tenure_labels)
    .values
)
# Approximate standard deviation for dti
dti_std = np.std(dti_means) if len(dti_means) > 1 else 1.0

# 1.e) Late-flag per Tenure (average + count)
df_late = pd.read_csv('Downloads/mortgage x late flag.csv')
# Columns: 'tenure label', 'late flag', 'Observations'
late_means = (
    df_late
    .set_index('tenure label')
    ['late flag']
    .reindex(tenure_labels)
    .values
)
late_counts = (
    df_late
    .set_index('tenure label')
    ['Observations']
    .reindex(tenure_labels)
    .values
)

# Placeholders
# Approximate standard deviation for dti
dti_std = 1.0
# Approximate standard deviation for late_flag
late_std = 1.0


# ─────────────────────────────────────────────────────────────────────────────
# 2) Prepare IPF on a 4-way table: Age × IncQuintile × Region × Tenure
# ─────────────────────────────────────────────────────────────────────────────

A = len(age_labels)
I = len(inc_labels)
R = len(region_labels)
T = len(tenure_labels)

# Observed margins (counts)
M_age_tenure = age_tenure_counts          # shape (A, T)
M_inc_tenure = inc_tenure_counts          # shape (I, T)
M_region_tenure = region_tenure_counts    # shape (R, T)

# Initialize seed 4D array with ones
X = np.ones((A, I, R, T), dtype=float)

# Run IPF for a fixed number of iterations
n_iters = 500
for _ in range(n_iters):
    # Match Age × Tenure margin
    current_age_tenure = X.sum(axis=(1, 2))            # shape (A, T)
    ratio_age_tenure = np.divide(M_age_tenure,
                                 current_age_tenure,
                                 where=current_age_tenure > 0)
    X *= ratio_age_tenure[:, np.newaxis, np.newaxis, :]

    # Match Income × Tenure margin
    current_inc_tenure = X.sum(axis=(0, 2))            # shape (I, T)
    ratio_inc_tenure = np.divide(M_inc_tenure,
                                 current_inc_tenure,
                                 where=current_inc_tenure > 0)
    X *= ratio_inc_tenure[np.newaxis, :, np.newaxis, :]

    # Match Region × Tenure margin
    current_region_tenure = X.sum(axis=(0, 1))         # shape (R, T)
    ratio_region_tenure = np.divide(M_region_tenure,
                                    current_region_tenure,
                                    where=current_region_tenure > 0)
    X *= ratio_region_tenure[np.newaxis, np.newaxis, :, :]

# Convert counts to joint probability
joint_prob = X / X.sum()  # shape (A, I, R, T)


# ─────────────────────────────────────────────────────────────────────────────
# 3) Define posterior function including DTI and late-flag likelihoods
# ─────────────────────────────────────────────────────────────────────────────

def normal_pdf(x, mean, std):
    """Gaussian PDF."""
    var = std * std
    return np.exp(-0.5 * ((x - mean) ** 2) / var) / (np.sqrt(2 * np.pi * var))

def posterior_tenure(age_group, inc_quintile, region, dti_value, late_value):
    """
    Compute P(tenure_label | age_group, inc_quintile, region, dti_value, late_value)
    via:
      P(tenure | age, inc, region) from IPF × P(dti | tenure) × P(late_flag | tenure).
    """
    try:
        a_idx = age_labels.index(age_group)
        i_idx = inc_labels.index(inc_quintile)
        r_idx = region_labels.index(region)
    except ValueError:
        raise KeyError("One of (age_group, inc_quintile, region) not found.")

    # Base probability from IPF
    base_probs = joint_prob[a_idx, i_idx, r_idx, :]  # shape (T,)

    # Likelihoods
    dti_likelihood = np.array([
        normal_pdf(dti_value, dti_means[t], dti_std) for t in range(T)
    ])
    late_likelihood = np.array([
        normal_pdf(late_value, late_means[t], late_std) for t in range(T)
    ])

    unnorm = base_probs * dti_likelihood * late_likelihood
    if unnorm.sum() == 0:
        # Fallback to IPF prior if all zero
        posterior = base_probs / base_probs.sum()
    else:
        posterior = unnorm / unnorm.sum()
    
    #return pd.Series(base_probs / base_probs.sum(), index=tenure_labels)
    return pd.Series(posterior, index=tenure_labels)


# ─────────────────────────────────────────────────────────────────────────────
# 4) Load 'robust_alternative.csv', sample tenure_label, and save
# ─────────────────────────────────────────────────────────────────────────────




# Load robust_alternative
df_alt = pd.read_csv('Downloads/robust_alternatives.csv')
dti_std = df_alt['dti'].std()
late_std = df_alt['any_late'].std()

# Expecting columns: 'age_group', 'income quintile', 'region', 'dti', 'any_late'
# Rename columns for consistency
df_alt = df_alt.rename(columns={
    'age_group': 'age_group',
    'income quintile': 'inc_quintile',
    'dti': 'dti',
    'any_late': 'late_flag',
    'region': 'region'
})

# For each row, compute posterior and sample tenure_label
np.random.seed(321)
tenure_samples = []
for _, row in df_alt.iterrows():
    post = posterior_tenure(
        age_group=row['age_group'],
        inc_quintile=row['inc_quintile'],
        region=row['region'],
        dti_value=row['dti'],
        late_value=row['late_flag']
    )
    # Sample one tenure label according to posterior
    tenure_choice = np.random.choice(tenure_labels, p=post.values)
    tenure_samples.append(tenure_choice)

# Add the sampled tenure_label column
df_alt['tenure_label'] = tenure_samples

df_alt = df_alt.rename(columns={
    'age_group': 'age_group',
    'inc_quintile': 'income quintile',
    'dti': 'dti',
    'late_flag': 'any_late',
    'region': 'region'
})


# Save to a new CSV
#df_alt.to_csv('Downloads/robust_alternative_with_tenure.csv', index=False)


In [43]:
df_alt = df_alt.drop(['housing_cost'], axis = 1)

KeyError: "['housing_cost'] not found in axis"

In [6]:
df_alt = df_alt.drop(['Unnamed: 0'], axis = 1)

In [7]:
df_alt

,income,incQ,dti,region,SOC_label,any_late,age_group,income quintile,tenure_label
0,47387.57,8,0.235342,South West,Professional occupations,0,Aged 35 to 49 years,4,Renter
1,66577.07,10,0.000000,East,"Managers, directors and senior officials",0,Aged 65 years and over,5,Own outright
2,33240.98,6,0.000000,South East,Professional occupations,0,Aged 50 to 64 years,3,Renter
3,41529.33,7,0.036065,South West,Professional occupations,1,Aged 35 to 49 years,4,Own outright
4,56176.13,9,0.000000,South East,Professional occupations,0,Aged 25 to 34 years,5,Renter
...,...,...,...,...,...,...,...,...,...
99995,32075.79,5,0.016850,South West,Skilled trades,1,Aged 35 to 49 years,3,Renter
99996,44287.21,8,0.000000,London,Professional occupations,1,Aged 35 to 49 years,4,Renter
99997,51767.80,9,0.000000,East,Professional occupations,0,Aged 50 to 64 years,5,Mortgage
99998,16043.91,2,0.000000,Wales,Professional occupations,0,Aged 35 to 49 years,1,Own outright


In [46]:
#df_alt.to_csv('Downloads/robustness_alternative_2.csv')

Check the breakdown on housing tenure labels from our alternative imputation

In [8]:
df_alt.groupby('tenure_label').agg({'income':'count'})

,income
tenure_label,
Mortgage,31302
Own outright,22690
Renter,46008


Similar code as per main file to model proportion of income spent on housing cost

In [9]:
from scipy.stats import beta

# ── 0) For reproducibility ────────────────────────────────────────────────
np.random.seed(42)

# ── 1) Load your customer data ────────────────────────────────────────────
# salary_df2 has cols: ['income_quintile','region','tenure_label', ...]
#salary_df2 = pd.read_csv('path/to/salary_df2.csv')

# ── 2) Load the two marginal tables ────────────────────────────────────────
# region table: region, tenure_label, prop_mean, frequency
reg_df = pd.read_csv('Downloads/housing cost x region.csv')

# income‐quintile table: income_quintile, tenure_label, prop_mean, frequency
inc_df = pd.read_csv('Downloads/housing cost x income_quintile.csv')

# ── 3) Method‐of‐moments → α, β for each table ──────────────────────────────
#   α = prop_mean * frequency,  β = (1−prop_mean)*frequency

reg_df['alpha_reg'] = reg_df['prop_mean'] * reg_df['frequency']
reg_df['beta_reg']  = (1 - reg_df['prop_mean']) * reg_df['frequency']

inc_df['alpha_inc'] = inc_df['prop_mean'] * inc_df['frequency']
inc_df['beta_inc']  = (1 - inc_df['prop_mean']) * inc_df['frequency']

# ── 4) Merge α,β into the main DataFrame ───────────────────────────────────
df2 = df_alt.merge(
    reg_df[['region','tenure_label','alpha_reg','beta_reg']],
    on=['region','tenure_label'],
    how='left'
).merge(
    inc_df[['income quintile','tenure_label','alpha_inc','beta_inc']],
    on=['income quintile','tenure_label'],
    how='left'
)

# ── 5) Compute posterior parameters ────────────────────────────────────────
df2['alpha_post'] = df2['alpha_reg'] + df2['alpha_inc']
df2['beta_post']  = df2['beta_reg']  + df2['beta_inc']

# ── 6) Sample proportion from Beta(α_post, β_post) ────────────────────────
def draw_prop(row):
    a, b = row['alpha_post'], row['beta_post']
    # fallback: if params are invalid, use the region‐only mean
    if pd.isna(a) or pd.isna(b) or a <= 0 or b <= 0:
        return row.get('prop_mean', np.nan)
    return beta.rvs(a, b)

df2['proportion_of_income_spent_on_housing_cost'] = df2.apply(draw_prop, axis=1)

# ── 7) (Optional) clean up intermediates ───────────────────────────────────
df2 = df2.drop(columns=[
    'alpha_reg','beta_reg','alpha_inc','beta_inc',
    'alpha_post','beta_post'
])

In [10]:
df2 = df2.fillna(0)
df2

,income,incQ,dti,region,SOC_label,any_late,age_group,income quintile,tenure_label,proportion_of_income_spent_on_housing_cost
0,47387.57,8,0.235342,South West,Professional occupations,0,Aged 35 to 49 years,4,Renter,0.200311
1,66577.07,10,0.000000,East,"Managers, directors and senior officials",0,Aged 65 years and over,5,Own outright,0.000000
2,33240.98,6,0.000000,South East,Professional occupations,0,Aged 50 to 64 years,3,Renter,0.225018
3,41529.33,7,0.036065,South West,Professional occupations,1,Aged 35 to 49 years,4,Own outright,0.000000
4,56176.13,9,0.000000,South East,Professional occupations,0,Aged 25 to 34 years,5,Renter,0.207660
...,...,...,...,...,...,...,...,...,...,...
99995,32075.79,5,0.016850,South West,Skilled trades,1,Aged 35 to 49 years,3,Renter,0.228205
99996,44287.21,8,0.000000,London,Professional occupations,1,Aged 35 to 49 years,4,Renter,0.256636
99997,51767.80,9,0.000000,East,Professional occupations,0,Aged 50 to 64 years,5,Mortgage,0.132579
99998,16043.91,2,0.000000,Wales,Professional occupations,0,Aged 35 to 49 years,1,Own outright,0.000000


In [11]:
df2['proportion_of_income_spent_on_housing_cost']=df2['proportion_of_income_spent_on_housing_cost'].fillna(0)

In [12]:
df_alt['proportion_of_income_spent_on_housing_cost']=df2['proportion_of_income_spent_on_housing_cost']

Save our dataframe as a csv file

In [13]:
df_alt.to_csv('Downloads/robustness_alternative_2.csv')